In [1]:
import numpy as np
import sys
sys.path.append("../")

import information_decomposition as pid
from intersection_information import intersection_information

## A couple of simple binary examples

In [2]:
# Parameter definitions for the examples
sigma = 0.5 # prior probability of s=1
rho = 0.1 # bit-flip probability for S->R channel
gamma = 0.1 # bit-flip probability for S->C channel where this exists

In [3]:
# Joint PDF table definitions for the examples

# example where SI is only due to signal correlations, as R<-S->C
P1 = np.zeros((2,2,2))
P1[0,0,0] = sigma*(1-rho)*(1-gamma)
P1[0,0,1] = sigma*(1-rho)*(gamma)
P1[0,1,0] = sigma*rho*(1-gamma)
P1[0,1,1] = sigma*rho*gamma
P1[1,0,0] = (1-sigma)*rho*gamma
P1[1,0,1] = (1-sigma)*rho*(1-gamma)
P1[1,1,0] = (1-sigma)*(1-rho)*(gamma)
P1[1,1,1] = (1-sigma)*(1-rho)*(1-gamma)

# example that should have nonzero intersection, as S->R==C
P2 = np.zeros((2,2,2))
P2[0,0,0] = sigma*(1-rho)
P2[0,0,1] = 0
P2[0,1,0] = 0
P2[0,1,1] = sigma*rho
P2[1,0,0] = (1-sigma)*rho
P2[1,0,1] = 0
P2[1,1,0] = 0
P2[1,1,1] = (1-sigma)*(1-rho)

In [4]:
# Compute and print intersection information, together with "non-normalised" and "signal correlations-only" SI values
print(intersection_information(P1))
print(intersection_information(P2))

(0.0, 0.31992237501539894, 0.31992237501539894)
(0.21108203139531984, 0.53100440641071878, 0.31992237501539894)


## Introducing IDTxl

In [5]:
import idtxl
from idtxl.estimators_pid import synergy_tartu

In [6]:
# my decomposition for the P2 problem above, as a sanity check
[SI, CI, UI_Y, UI_Z] = pid.decomposition(P2)
print(SI, CI, UI_Y, UI_Z)

0.531004406411 0.0 0.0 0.0


In [7]:
# convert definition of P2 for tartu
P = dict()
for x in [0,1]:
    for y in [0,1]:
        for z in [0,1]:
            P[(x,y,z)] = P2[x,y,z]
print(P)

{(0, 1, 1): 0.050000000000000003, (0, 0, 0): 0.45000000000000001, (1, 1, 0): 0.0, (1, 0, 0): 0.050000000000000003, (0, 1, 0): 0.0, (0, 0, 1): 0.0, (1, 1, 1): 0.45000000000000001, (1, 0, 1): 0.0}


In [8]:
[the_p, feas, kkt, wCI, wSI, wUI_Y, wUI_Z] = synergy_tartu.solve_PDF(pdf=P)
print(wSI, wCI, wUI_Y, wUI_Z)

0.5310044058278642 5.82854542408e-10 0.0 0.0


## Example: dice

In [12]:
aa=np.int(0);

lstep = 0.05
l=0
l_n = np.int(np.ceil(1/lstep)+1)

I_shar=np.zeros(l_n);
I_uny=np.zeros(l_n);
I_unz=np.zeros(l_n);
I_syn=np.zeros(l_n);

I_shar_z=np.zeros(l_n);
I_uny_z=np.zeros(l_n);
I_unz_z=np.zeros(l_n);
I_syn_z=np.zeros(l_n);

#Y and Z are the dice, X is the weighted sum of the two


# summed-dice dimensions
dimy=np.int(6);
dimz=np.int(6);
 
alpha=np.int(1); # relative ratio between the sum coefficients for summing the dice
 
dimx=np.int(np.ceil(dimy+dimz*alpha-1-alpha)+1);



#make X the new target
dimx_x=dimz;
dimy_x=dimy;
dimz_x=dimx;


while aa<=np.ceil(1/lstep): # loop over different correlations between the dice

    
    p_yz = np.zeros((dimy,dimz))
    
    # setting correlations between the dice
    for yy in range(0,dimy):
        for zz in range(0,dimz):
            p_yz[yy,zz]=l/36+(1-l)/6*np.float(zz==yy)
    
    p_yz=p_yz/p_yz.sum() # p(y,z), joint probability of the two dice alone

    p_x_yz=np.zeros((dimx,dimy,dimz)) # p(x|y,z)

    #build p(x|y,z) analytically
    for xx in range(0,dimx):
        for yy in range(0,dimy):
            for zz in range(0,dimz):
                if xx==np.ceil(zz+alpha*yy)-alpha:
                    p_x_yz[xx,yy,zz]=1
   
    p=np.zeros((dimx,dimy,dimz))

    #build p analytically
    for xx in range(0,dimx):
        for yy in range(0,dimy):
            for zz in range(0,dimz):
                p[xx,yy,zz]=p_x_yz[xx,yy,zz]*p_yz[yy,zz]

    p=p/p.sum() #final normalization

    # adapt definitions to those expected by tartu
    P = dict()
    P_z = dict()
    for x in range(0,dimx):
        for y in range(0,dimy):
            for z in range(0,dimz):
                P[(x,y,z)] = p[x,y,z]
                P_z[(z,y,x)] = p[x,y,z]

    print('\nLambda: {:.2f}'.format(l))
    %time [_, _, _, I_syn[aa],I_shar[aa],I_uny[aa],I_unz[aa]] = synergy_tartu.solve_PDF(pdf=P)
    #%time [_, _, _, I_syn_z[aa],I_shar_z[aa],I_uny_z[aa],I_unz_z[aa]] = synergy_tartu.solve_PDF(pdf=P_z)
    

    aa+=1
    l+=lstep
    


Lambda: 0.00
CPU times: user 6.28 s, sys: 12.8 s, total: 19.1 s
Wall time: 5.69 s

Lambda: 0.05
CPU times: user 7.27 s, sys: 15.4 s, total: 22.7 s
Wall time: 6.66 s

Lambda: 0.10
CPU times: user 6.94 s, sys: 14.6 s, total: 21.6 s
Wall time: 6.13 s

Lambda: 0.15
CPU times: user 6.44 s, sys: 12.6 s, total: 19.1 s
Wall time: 5.42 s

Lambda: 0.20
CPU times: user 6.58 s, sys: 13.4 s, total: 20 s
Wall time: 5.64 s

Lambda: 0.25
CPU times: user 6.55 s, sys: 12.5 s, total: 19.1 s
Wall time: 5.38 s

Lambda: 0.30
CPU times: user 6.54 s, sys: 12.7 s, total: 19.2 s
Wall time: 5.43 s

Lambda: 0.35
CPU times: user 6.39 s, sys: 12.3 s, total: 18.7 s
Wall time: 5.31 s

Lambda: 0.40
CPU times: user 7.56 s, sys: 15.7 s, total: 23.3 s
Wall time: 6.74 s

Lambda: 0.45
CPU times: user 7.05 s, sys: 14.3 s, total: 21.3 s
Wall time: 6.02 s

Lambda: 0.50
CPU times: user 6.92 s, sys: 13.8 s, total: 20.7 s
Wall time: 5.84 s

Lambda: 0.55
CPU times: user 8.44 s, sys: 16.7 s, total: 25.2 s
Wall time: 7.03 s

Lambd

In [13]:
print(I_syn)

[ -1.23092603e-09   3.62954080e-01   6.35419338e-01   8.70101129e-01
   1.07845940e+00   1.26601621e+00   1.43605386e+00   1.59074580e+00
   1.73162538e+00   1.85981548e+00   1.97615336e+00   2.08126285e+00
   2.17559714e+00   2.25946294e+00   2.33303142e+00   2.39633784e+00
   2.44926910e+00   2.49153578e+00   2.52262036e+00   2.54168336e+00
   2.54738616e+00]


In [14]:
print(I_shar)

[ 2.32192809  2.14829493  2.00221035  1.86891262  1.74519403  1.62951543
  1.5209715   1.41897497  1.32312654  1.23315213  1.14886976  1.07017184
  0.99701649  0.9294252   0.86748589  0.81136164  0.76130686  0.71769419
  0.6810595   0.65217958  0.63221598]
